<a href="https://testpolly.elucidata.io/manage/workspaces?action=open_polly_notebook&source=github&path=path_place_holder&kernel=elucidata/Python 3.10&machine=medium" target="_parent"><img src="https://elucidatainc.github.io/PublicAssets/open_polly.svg" alt="Open in Polly"/></a>


# Welcome to Polly Python3 Notebook.

In [1]:
!sudo pip3 install https://elucidatainc.github.io/PublicAssets/builds/polly-python/tests/testpolly/polly_python-0.3.2_ux_gaps-py3-none-any.whl

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
     |████████████████████████████████| 138 kB 18.0 MB/s eta 0:00:01


  Attempting uninstall: polly-validator
    Found existing installation: polly-validator 0.0.1
    Uninstalling polly-validator-0.0.1:
      Successfully uninstalled polly-validator-0.0.1
  Attempting uninstall: polly-python
    Found existing installation: polly-python 0.3.0
    Uninstalling polly-python-0.3.0:
      Successfully uninstalled polly-python-0.3.0
You should consider upgrading via the '/usr/local/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
import os
from polly.auth import Polly
AUTH_TOKEN=(os.environ['POLLY_REFRESH_TOKEN']) # Obtain authentication tokens
Polly.auth(AUTH_TOKEN)

In [3]:
!polly files sync -s "polly://metadata/" -d "metadata/" -y

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Success: Installed optional packages!
Refreshing session...
Session refreshed!
Completed 6.4 KiB/6.4 KiB (8.0 KiB/s) with 1 file(s) remainingemainingSync complete!


In [4]:
!polly files sync -s "polly://data/" -d "data/" -y

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Completed 66.8 KiB/100.0 KiB (85.4 KiB/s) with 2 file(s) remainingCompleted 100.0 KiB/100.0 KiB (120.8 KiB/s) with 1 file(s) remainingSync complete!


In [5]:
from polly.validation import Validation
validation = Validation()

Retrieving ontological names...


/usr/local/lib/python3.10/site-packages/polly_validator/downloader/get_valid_names.py:46: UnicodeWarning: unsound encoding, assuming utf-8 (99% confidence)
  diseases = pt.Ontology(f'https://raw.githubusercontent.com/bioinfo-el/ontologies/'


Getting predefined values for Data Type and Data Source...
Retrieved Valid Data Types
Retrieved Valid Data Sources


# Indexing Configs Put Programmatically

### Putting ingestion_configs in the metadata file are manual right now

### If somebody wants to put the ingestion_configs programatically -> the below functionality will help users to achieve that

### Through the function -> users can put the right structure of  ingestion_configs in the metadata file

### If users want to change the ingestion_configs values in the dict -> that can also be done 

### This functionality will be mostly be helpful in bulk validation and ingestion where users want to have same ingestion_configs for each file.

In [6]:
# step 1 -> get ingestion config
ingestion_configs = validation.get_ingestion_configs()
print("---ingestion configs----")
print(ingestion_configs)

---ingestion configs----
{'file_metadata': True, 'col_metadata': True, 'row_metadata': True, 'data_required': True, 'validation_check': {'dataset': {'validate': True, 'scope': 'advanced', 'force_ingest': False}, 'sample': {'validate': True, 'scope': 'advanced', 'force_ingest': False}}}


In [7]:
# step 2 -> put them in the metadata json file of dataset
# loading data

# step 2.1 -> define folder paths and files to be excluded
source_folder_path_data = "/import/data"
source_folder_metadata = "/import/metadata"
source_folder_path = {"data":source_folder_path_data, "metadata":source_folder_metadata}

source_folder_path_metadata = source_folder_path["metadata"]

VALIDATION_STATUS_FILE_NAME = "validation_status.txt"


import os
from pathlib import Path

# step 2.2 -> create list of metadata files to be edited
metadata_directory = os.fsencode(source_folder_path_metadata)
metadata_file_list = []
for file in os.listdir(metadata_directory):
    file = file.decode("utf-8")
    # skip hidden files
    # skip the validation_status.txt
    # in the metadata folder
    if not file.startswith(".") and file != VALIDATION_STATUS_FILE_NAME:
        metadata_file_list.append(file)
        


# step 2.3 -> load the file and add the ingestion configs
for file in metadata_file_list:
    file_path = str(Path(source_folder_path_metadata) / Path(os.fsdecode(file)))
    # read data from file
    with open(file_path, "r") as metadata_file:
        metadata_file_dict = json.load(metadata_file)
        metadata_file_dict["__index__"] = ingestion_configs
    
    # write data to file
    with open(file_path, "w") as metadata_file:
        json.dump(metadata_file_dict, metadata_file, indent=4)

In [8]:
import pprint
# read the file again to check if ingestion_configs added
for file in metadata_file_list:
    file_path = str(Path(source_folder_path_metadata) / Path(os.fsdecode(file)))

    with open(file_path, "r") as metadata_file:
        metadata_file_dict_edited = json.load(metadata_file)
        print(f"----edited file dict of file: {file}")
        print("\n")
        pprint.pprint(metadata_file_dict_edited, sort_dicts=False)
    
    print("\n")
    

----edited file dict of file: ACBC_MSKCC_2015_Copy_Number_AdCC10T.json


{'__index__': {'file_metadata': True,
               'col_metadata': True,
               'row_metadata': True,
               'data_required': True,
               'validation_check': {'dataset': {'validate': True,
                                                'scope': 'advanced',
                                                'force_ingest': False},
                                    'sample': {'validate': True,
                                               'scope': 'advanced',
                                               'force_ingest': False}}},
 'curated_file_type': None,
 'curated_organism': ['Homo sapiens'],
 'src_uri': 'polly:data://cbioportal/data/ACBC_MSKCC_2015/CNV/ACBC_MSKCC_2015_Copy_Number_AdCC11T.gct',
 'total_num_samples': 1,
 'curated_disease_type': 'Invasive Breast Carcinoma',
 'sample_id': 'AdCC11T',
 'description': 'ACBC_MSKCC_2015 Copy Number Variation dataset for sample id - '
        

In [9]:
# lib_repo_dev repo on testpolly

# schema config is totally optional
# just for demo purposes put here
schema_config = {}
schema_config["source"] = "all"
schema_config["datatype"] = "all"


repo_id = 1673847977346
error_df, status_dict = validation.validate_datasets(repo_id, source_folder_path, schema_config)
print("----------error_df-----------")
print(error_df)
print("-----status dict-----")
pprint.pprint(status_dict)

Combining Metadata Files for Validation: 100%|██████████| 3/3 [00:00<00:00, 3672.77it/s]


--validate_val: True
--validate_val: True
--validate_val: True
╒════╤═════════════════════════════════════╤═══════════════════════════╤═══════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════╕
│    │ dataset_id                          │ Field                     │ Original Name         │ Error Message                                                                                 │          Repo │
╞════╪═════════════════════════════════════╪═══════════════════════════╪═══════════════════════╪═══════════════════════════════════════════════════════════════════════════════════════════════╪═══════════════╡
│  0 │ ACBC_MSKCC_2015_Copy_Number_AdCC11T │ platform                  │ platform              │ field required                                                                                │ 1673847977346 │
├────┼─────────────────────────────────────┼───────────────────────────┼───────────────────────┼─────

Generating Status File of Validation: 100%|██████████| 3/3 [00:00<00:00, 4458.86it/s]


----------error_df-----------
                             dataset_id                      Field  \
0   ACBC_MSKCC_2015_Copy_Number_AdCC11T                   platform   
1   ACBC_MSKCC_2015_Copy_Number_AdCC11T                 timestamp_   
2   ACBC_MSKCC_2015_Copy_Number_AdCC11T              subseries_ids   
3   ACBC_MSKCC_2015_Copy_Number_AdCC11T  data_matrix_available_new   
4   ACBC_MSKCC_2015_Copy_Number_AdCC11T            missing_samples   
5   ACBC_MSKCC_2015_Copy_Number_AdCC11T                    summary   
6   ACBC_MSKCC_2015_Copy_Number_AdCC11T                drug_smiles   
7   ACBC_MSKCC_2015_Copy_Number_AdCC11T                     author   
8   ACBC_MSKCC_2015_Copy_Number_AdCC11T               curated_gene   
9   ACBC_MSKCC_2015_Copy_Number_AdCC11T                   abstract   
10  ACBC_MSKCC_2015_Copy_Number_AdCC11T             curated_strain   
11  ACBC_MSKCC_2015_Copy_Number_AdCC11T             overall_design   
12  ACBC_MSKCC_2015_Copy_Number_AdCC11T                 is_c

In [10]:
# changing the ingestion_dict to put validate -> False
# Structure of index in the metadata file

# this is just for demo purposes so that users can visualise the structure
index_structure = {
    "__index__": {
        "file_metadata": True,
        "col_metadata": True,
        "row_metadata": True,
        "data_required": True,
        "validation_check": {
            "dataset": {
                "validate": True, 
                "scope": "advanced", 
                "force_ingest": False
            }, 
            "sample": {
                "validate": True, 
                "scope": "advanced", 
                "force_ingest": False
            }
        }
  }
}

In [11]:
# edit the ingestion_dict to put validate -> False 
# for all the files

# step 2.3 -> load the file and add the ingestion configs
for file in metadata_file_list:
    file_path = str(Path(source_folder_path_metadata) / Path(os.fsdecode(file)))
    # read data from file
    with open(file_path, "r") as metadata_file:
        metadata_file_dict = json.load(metadata_file)
        metadata_file_dict["__index__"] = ingestion_configs
        # editing ingestion configs to put validate to False
        metadata_file_dict["__index__"]["validation_check"]["dataset"]["validate"] = False
        metadata_file_dict["__index__"]["validation_check"]["sample"]["validate"] = False
    
    # write data to file
    with open(file_path, "w") as metadata_file:
        json.dump(metadata_file_dict, metadata_file, indent=4)


In [12]:
import pprint
# read the file again to check if ingestion_configs added
for file in metadata_file_list:
    file_path = str(Path(source_folder_path_metadata) / Path(os.fsdecode(file)))

    with open(file_path, "r") as metadata_file:
        metadata_file_dict_edited = json.load(metadata_file)
        print(f"----edited file dict of file: {file}")
        print("\n")
        pprint.pprint(metadata_file_dict_edited, sort_dicts=False)
    
    print("\n")

----edited file dict of file: ACBC_MSKCC_2015_Copy_Number_AdCC10T.json


{'__index__': {'file_metadata': True,
               'col_metadata': True,
               'row_metadata': True,
               'data_required': True,
               'validation_check': {'dataset': {'validate': False,
                                                'scope': 'advanced',
                                                'force_ingest': False},
                                    'sample': {'validate': False,
                                               'scope': 'advanced',
                                               'force_ingest': False}}},
 'curated_file_type': None,
 'curated_organism': ['Homo sapiens'],
 'src_uri': 'polly:data://cbioportal/data/ACBC_MSKCC_2015/CNV/ACBC_MSKCC_2015_Copy_Number_AdCC11T.gct',
 'total_num_samples': 1,
 'curated_disease_type': 'Invasive Breast Carcinoma',
 'sample_id': 'AdCC11T',
 'description': 'ACBC_MSKCC_2015 Copy Number Variation dataset for sample id - '
      

In [13]:
# validating the files where validate is False -> it will give warning
# lib_repo_dev repo on testpolly

# schema config is totally optional
# just for demo purposes put here
schema_config = {}
schema_config["source"] = "all"
schema_config["datatype"] = "all"


# WARNINGS ARE NOT GETTING PRINTED HERE -> BUT IT IS GETTING PRINTED ON LOCAL

repo_id = 1673847977346
validation.validate_datasets(repo_id, source_folder_path, schema_config)


Combining Metadata Files for Validation: 100%|██████████| 3/3 [00:00<00:00, 3575.71it/s]


--validate_val: False


--validate_val: False


--validate_val: False


Validation not executed on any of the metadata files.  Possible reasons to help you troubleshoot :-
All the metadata file passed may have `validate flag in the `__index__` dict to be false. Plese set them to true for the files which needs to be validated.


In [ ]:
# IF wrong values put in the validation_config keys
# editing the dict and putting wrong values

In [14]:
# edit the ingestion_dict to put validate -> temp (undesirable value) 
# for all the files

# step 2.3 -> load the file and add the ingestion configs
for file in metadata_file_list:
    file_path = str(Path(source_folder_path_metadata) / Path(os.fsdecode(file)))
    # read data from file
    with open(file_path, "r") as metadata_file:
        metadata_file_dict = json.load(metadata_file)
        metadata_file_dict["__index__"] = ingestion_configs
        # editing ingestion configs to put validate to False
        metadata_file_dict["__index__"]["validation_check"]["dataset"]["validate"] = "temp"
        metadata_file_dict["__index__"]["validation_check"]["sample"]["validate"] = "temp"
    
    # write data to file
    with open(file_path, "w") as metadata_file:
        json.dump(metadata_file_dict, metadata_file, indent=4)

In [15]:
import pprint
# read the file again to check if edited ingestion_configs added
for file in metadata_file_list:
    file_path = str(Path(source_folder_path_metadata) / Path(os.fsdecode(file)))

    with open(file_path, "r") as metadata_file:
        metadata_file_dict_edited = json.load(metadata_file)
        print(f"----edited file dict of file: {file}")
        print("\n")
        pprint.pprint(metadata_file_dict_edited, sort_dicts=False)
    
    print("\n")

----edited file dict of file: ACBC_MSKCC_2015_Copy_Number_AdCC10T.json


{'__index__': {'file_metadata': True,
               'col_metadata': True,
               'row_metadata': True,
               'data_required': True,
               'validation_check': {'dataset': {'validate': 'temp',
                                                'scope': 'advanced',
                                                'force_ingest': False},
                                    'sample': {'validate': 'temp',
                                               'scope': 'advanced',
                                               'force_ingest': False}}},
 'curated_file_type': None,
 'curated_organism': ['Homo sapiens'],
 'src_uri': 'polly:data://cbioportal/data/ACBC_MSKCC_2015/CNV/ACBC_MSKCC_2015_Copy_Number_AdCC11T.gct',
 'total_num_samples': 1,
 'curated_disease_type': 'Invasive Breast Carcinoma',
 'sample_id': 'AdCC11T',
 'description': 'ACBC_MSKCC_2015 Copy Number Variation dataset for sample id - '
    

In [16]:
# validating the files where validate is False -> it will give warning
# lib_repo_dev repo on testpolly

# schema config is totally optional
# just for demo purposes put here
schema_config = {}
schema_config["source"] = "all"
schema_config["datatype"] = "all"


# WARNINGS ARE NOT GETTING PRINTED HERE -> BUT IT IS GETTING PRINTED ON LOCAL

repo_id = 1673847977346
validation.validate_datasets(repo_id, source_folder_path, schema_config)

Combining Metadata Files for Validation:   0%|          | 0/3 [00:00<?, ?it/s]


--validate_val: temp


paramException: paramException (parameter error): validate key is set to incorrect type in ACBC_MSKCC_2015_Copy_Number_AdCC10T.json. Only boolean allowed

In [17]:
# edit the ingestion_dict and put wrong values for scope key
# it will raise an error

for file in metadata_file_list:
    file_path = str(Path(source_folder_path_metadata) / Path(os.fsdecode(file)))
    # read data from file
    with open(file_path, "r") as metadata_file:
        metadata_file_dict = json.load(metadata_file)
        metadata_file_dict["__index__"] = ingestion_configs
        # editing ingestion configs to put validate to true
        metadata_file_dict["__index__"]["validation_check"]["dataset"]["validate"] = True
        metadata_file_dict["__index__"]["validation_check"]["sample"]["validate"] = True

        # change the scope to incorrect val
        metadata_file_dict["__index__"]["validation_check"]["dataset"]["scope"] = "mkcneknce"
        metadata_file_dict["__index__"]["validation_check"]["sample"]["scope"] = "nkneknken"
    
    # write data to file
    with open(file_path, "w") as metadata_file:
        json.dump(metadata_file_dict, metadata_file, indent=4)


import pprint
# read the file again to check if ingestion_configs added
for file in metadata_file_list:
    file_path = str(Path(source_folder_path_metadata) / Path(os.fsdecode(file)))

    with open(file_path, "r") as metadata_file:
        metadata_file_dict_edited = json.load(metadata_file)
        print(f"----edited file dict of file: {file}")
        print("\n")
        pprint.pprint(metadata_file_dict_edited, sort_dicts=False)
    
    print("\n")

----edited file dict of file: ACBC_MSKCC_2015_Copy_Number_AdCC10T.json


{'__index__': {'file_metadata': True,
               'col_metadata': True,
               'row_metadata': True,
               'data_required': True,
               'validation_check': {'dataset': {'validate': True,
                                                'scope': 'mkcneknce',
                                                'force_ingest': False},
                                    'sample': {'validate': True,
                                               'scope': 'nkneknken',
                                               'force_ingest': False}}},
 'curated_file_type': None,
 'curated_organism': ['Homo sapiens'],
 'src_uri': 'polly:data://cbioportal/data/ACBC_MSKCC_2015/CNV/ACBC_MSKCC_2015_Copy_Number_AdCC11T.gct',
 'total_num_samples': 1,
 'curated_disease_type': 'Invasive Breast Carcinoma',
 'sample_id': 'AdCC11T',
 'description': 'ACBC_MSKCC_2015 Copy Number Variation dataset for sample id - '
      

In [18]:

# validating the files where scope is not the desirable value -> raise error
# lib_repo_dev repo on testpolly

# schema config is totally optional
# just for demo purposes put here
schema_config = {}
schema_config["source"] = "all"
schema_config["datatype"] = "all"


repo_id = 1673847977346
validation.validate_datasets(repo_id, source_folder_path, schema_config)

Combining Metadata Files for Validation:   0%|          | 0/3 [00:00<?, ?it/s]


--validate_val: True


paramException: paramException (parameter error): scope key is set to incorrect type/val in ACBC_MSKCC_2015_Copy_Number_AdCC10T.json. Only ['basic', 'advanced'] allowed.